<a href="https://colab.research.google.com/github/daman1209arora/cnn-mnist/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
from torch import nn, optim
import torch
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [4]:
train_batch_size = 64
test_batch_size = 1000

In [5]:

mnist = MNIST(download=True, train=True, root=".").train_data.float()

data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])

train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                          batch_size=train_batch_size, shuffle=True)

test_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                        batch_size=test_batch_size, shuffle=False)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [6]:
import torchvision.models as models

In [7]:
class MnistResNet(nn.Module):
  def __init__(self):
        super(MnistResNet, self).__init__()
        self.baseline = models.resnet18(pretrained=True)
        self.last_fc = nn.Linear(in_features = 1000, out_features = 10)
  def forward(self, x):
    x = self.baseline.forward(x)
    return F.softmax(self.last_fc(x))



In [18]:
import torch.optim
import torchvision

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MnistResNet().to(device)

criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
for epoch in range(5):
  print(epoch)
  e_loss = 0.0
  for i,data in enumerate(train_loader):
    input,label = data
    input, label = input.to(device), label.to(device)
    input = torch.cat([input, input, input], dim=1)
    optimizer.zero_grad()
    output = model(input)
    loss = criteria(output,label)
    loss.backward()
    optimizer.step()

    e_loss+=loss.item()


  with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        images = torch.cat([images, images, images], dim=1)
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
  print("Accuracy on the test set:", 100 * correct / total)
  
  print("loss %f",e_loss)
  e_loss=0
print("trained")

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


Accuracy on the test set: 9.25
loss %f 2223.7811980247498
1
Accuracy on the test set: 9.14
loss %f 2223.8095297813416
2


KeyboardInterrupt: ignored

In [21]:
data_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)), torchvision.transforms.ToTensor()])
mnist_data = torchvision.datasets.MNIST("./data", download=True, transform=data_transform)
size = len(mnist_data)

train_data, test_data = torch.utils.data.random_split(mnist_data, [int(0.9 * size), int(0.1 * size)])
train_data_loader = torch.utils.data.DataLoader(train_data, 32, shuffle=True)

total_loss = 0
for epoch in range(5):
    for i, data in enumerate(train_data_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        images = torch.cat([images, images, images], dim=1)
        #images = torch.flatten(images, start_dim=1)
        optimizer.zero_grad()

        output = model(images)
        loss = criteria(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 
        if(i % 100 == 99):
            print("After epoch {}, mini-batch {}, loss: {}".format(epoch + 1, i + 1, total_loss))
            correct = 0
            total = 0

            test_data_loader = torch.utils.data.DataLoader(test_data, 5, shuffle=True)
            with torch.no_grad():
                for i, data in enumerate(test_data_loader, 0):
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    images = torch.cat([images, images, images], dim=1)
                    output = model(images)
                    _, predicted = torch.max(output.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                
            print("Accuracy on the test set:", 100 * correct / total)
            total_loss = 0
    total_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


After epoch 1, mini-batch 100, loss: 237.3650884628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 200, loss: 237.5838384628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 300, loss: 236.6150884628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 400, loss: 236.0213384628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 500, loss: 237.3963384628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 600, loss: 236.6150884628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 700, loss: 237.7400884628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 800, loss: 237.6775884628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 900, loss: 237.4588384628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 1000, loss: 237.4588384628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 1100, loss: 236.7088384628296
Accuracy on the test set: 8.8
After epoch 1, mini-batch 1200, loss: 237.0213384628296
Accurac

KeyboardInterrupt: ignored

In [11]:

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device) , labels.to(device)
        images = torch.cat([images,images,images],dim=1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy ',(100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


Accuracy  9.58


In [14]:
def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    # just an utility printing function
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")


In [ ]:
#  params you need to specify:
epochs = 5
train_loader, val_loader = # put your data loader here
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, I've used Adadelta, as it wokrs well without any magic numbers
optimizer = optim.Adadelta(model.parameters())

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

losses = []
batches = len(train_loader)
val_batches = len(val_loader)

# loop for every epoch (training + evaluation)
for epoch in range(epochs):
    total_loss = 0

    # progress bar (works in Jupyter notebook too!)
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

    # ----------------- TRAINING  -------------------- 
    # set model to training
    
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        
        # training step for single batch
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)
        loss.backward()
        optimizer.step()

        # getting training quality data
        current_loss = loss.item()
        total_loss += current_loss

        # updating progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # ----------------- VALIDATION  ----------------- 
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    # set model to evaluating (testing)
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)

            outputs = model(X) # this get's the prediction from the network

            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
            
            # calculate P/R/F1/A metrics for batch
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
          
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches) # for plotting learning curve
print(f"Training time: {time.time()-start_ts}s")